# Events tablica

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime
from scipy.stats import shapiro 
from scipy.stats import lognorm
from scipy.stats import kstest,norm
from scipy.stats import lognorm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from sklearn.preprocessing import normalize
import warnings
warnings.filterwarnings("ignore")

In [14]:
f_eve = Path.cwd()
f_eve = str(f_eve) + "/events.csv"
events = pd.read_csv(f_eve)
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   holiday_name      4 non-null      object
 1   date_from         4 non-null      object
 2   date_to           4 non-null      object
 3   country_of_event  4 non-null      object
 4   repeating_date    4 non-null      int64 
 5   yearly_repeating  4 non-null      int64 
dtypes: int64(2), object(4)
memory usage: 324.0+ bytes


In [18]:
events["date_from"] = pd.to_datetime(events["date_from"],format="%d-%m-%Y")
events["date_to"] = pd.to_datetime(events["date_to"],format="%d-%m-%Y")
events["country_of_event"] = events["country_of_event"].astype("string")
events["holiday_name"] = events["holiday_name"].astype("string")
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   holiday_name      4 non-null      string        
 1   date_from         4 non-null      datetime64[ns]
 2   date_to           4 non-null      datetime64[ns]
 3   country_of_event  4 non-null      string        
 4   repeating_date    4 non-null      string        
 5   yearly_repeating  4 non-null      int64         
dtypes: datetime64[ns](2), int64(1), string(3)
memory usage: 324.0 bytes


In [19]:
num_years = 1

# Function to duplicate rows for repeating events
# Function to duplicate rows for repeating events
def duplicate_rows_for_repeats(events, num_years):
    new_rows = []
    for _, row in events.iterrows():
        # Access the 'repeat' column by its position
        if row.iloc[-1] == 'da':  # Assuming 'repeat' is the last column
            for i in range(1, num_years + 1):
                new_row = row.copy()
                new_row['date_from'] = pd.to_datetime(row['date_from']) + pd.DateOffset(years=i)
                new_row['date_to'] = pd.to_datetime(row['date_to']) + pd.DateOffset(years=i)
                new_rows.append(new_row)
        else:
            new_rows.append(row)
    return pd.DataFrame(new_rows)

# Add rows for repeating events
new_df = duplicate_rows_for_repeats(events, num_years)


# Concatenate the original DataFrame with the new DataFrame containing repeated events
final_df = pd.concat([events, new_df], ignore_index=True)

print(final_df)

  holiday_name  date_from    date_to country_of_event repeating_date  \
0  Nova_godina 2008-12-31 2009-01-01              HRV              1   
1        Bozic 2008-12-23 2008-12-25              HRV              1   
2     Uskrs-08 2008-03-23 2008-03-24              HRV              0   
3     Uskrs-09 2009-04-12 2009-04-13              HRV              0   
4  Nova_godina 2008-12-31 2009-01-01              HRV              1   
5        Bozic 2008-12-23 2008-12-25              HRV              1   
6     Uskrs-08 2008-03-23 2008-03-24              HRV              0   
7     Uskrs-09 2009-04-12 2009-04-13              HRV              0   

   yearly_repeating  
0                 1  
1                 1  
2                 1  
3                 1  
4                 1  
5                 1  
6                 1  
7                 1  
